In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

data = pd.read_csv('heart_failure_clinical_records_dataset.csv')
# Split the dataset into features (X) and target (y)
X = data.drop(columns=['DEATH_EVENT'])
y = data['DEATH_EVENT']

# Split the dataset into training and testing sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


# Fit the Logistic Regression model
#log_reg = LogisticRegression(random_state=42)
model = RandomForestClassifier(random_state=43)
model.fit(X_train,y_train)
temp = pd.DataFrame(model.predict_proba(X_train).tolist(), columns=model.classes_)
temp['max_prob'] = temp.max(axis=1)

#log_reg.fit(X_train_scaled, y_train)

# Predict on the test set
#y_pred = log_reg.predict(X_test_scaled)
y_rf = model.predict(X_test)
# Calculate the accuracy score
#accuracy = accuracy_score(y_test, y_pred)

idx = temp[temp.max_prob >= .9].index
accuracy2 = accuracy_score(y_test.iloc[idx], y_rf[idx])


accuracy2


IndexError: positional indexers are out-of-bounds

In [8]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Load the dataset
data = pd.read_csv('heart_failure_clinical_records_dataset.csv')

# Split the dataset into features (X) and target (y)
X = data.drop(columns=['DEATH_EVENT'])
y = data['DEATH_EVENT']

# Split the dataset into training and testing sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Fit the Random Forest Classifier model
model = RandomForestClassifier(random_state=43)
model.fit(X_train_scaled, y_train)

# Predict probabilities on the test set
probs = model.predict_proba(X_test_scaled)

# Convert the predicted probabilities to a DataFrame
probs_df = pd.DataFrame(probs, columns=model.classes_)

# Add a column for the maximum predicted probability
probs_df['max_prob'] = probs_df.max(axis=1)

# Get the predicted class using the maximum probability
probs_df['pred_class'] = model.predict(X_test_scaled)

# Set a probability threshold (e.g., 0.9 for high-confidence predictions)
threshold = 0.9275

# Filter rows where the maximum probability is above the threshold
high_confidence_idx = probs_df[probs_df['max_prob'] >= threshold].index
data2 = pd.DataFrame(data)
newData = data2.loc[high_confidence_idx]
newData = data2.drop(data2.index.difference(high_confidence_idx))

# Get the high-confidence predictions and their corresponding true labels
high_confidence_preds = probs_df.loc[high_confidence_idx, 'pred_class']
high_confidence_true = y_test.iloc[high_confidence_idx]

# Calculate the accuracy for high-confidence predictions
accuracy_high_confidence = accuracy_score(high_confidence_true, high_confidence_preds)


accuracy_high_confidence , len(high_confidence_idx)/len(y_test)



(1.0, 0.2833333333333333)

#Heart Failure Prediction Model

##Loading Data

In [7]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.ensemble import RandomForestClassifier
# Load the data
data = pd.read_csv('heart_failure_clinical_records_dataset.csv')
data

,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT
0,75.0,0,582,0,20,1,265000.00,1.9,130,1,0,4,1
1,55.0,0,7861,0,38,0,263358.03,1.1,136,1,0,6,1
2,65.0,0,146,0,20,0,162000.00,1.3,129,1,1,7,1
3,50.0,1,111,0,20,0,210000.00,1.9,137,1,0,7,1
4,65.0,1,160,1,20,0,327000.00,2.7,116,0,0,8,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
294,62.0,0,61,1,38,1,155000.00,1.1,143,1,1,270,0
295,55.0,0,1820,0,38,0,270000.00,1.2,139,0,0,271,0
296,45.0,0,2060,1,60,0,742000.00,0.8,138,0,0,278,0
297,45.0,0,2413,0,38,0,140000.00,1.4,140,1,1,280,0


##Splitting Data

In [2]:
# Separate features (X) and target (y)
X = data.drop(columns=['DEATH_EVENT'])
y = data['DEATH_EVENT']

# Split the data into training and testing sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


##Standardize the features

In [3]:
# Standardize the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


##Define hyperparameter grid for Logistic Regression

In [5]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression

# Define hyperparameter grid for Logistic Regression
param_grid = [
    {'C': [0.001, 0.01, 0.1, 1, 10, 100], 'penalty': ['l2'], 'solver': ['lbfgs', 'sag']},
    {'C': [0.001, 0.01, 0.1, 1, 10, 100], 'penalty': ['l1', 'l2'], 'solver': ['liblinear', 'saga']}
]

# Create a GridSearchCV object with LogisticRegression and optimized scoring
log_reg_cv = GridSearchCV(LogisticRegression(random_state=42), param_grid=param_grid, cv=5, scoring='roc_auc')

# Fit the model
log_reg_cv.fit(X_train, y_train)

c:\Users\Abhi dobariya\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
c:\Users\Abhi dobariya\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://sci

GridSearchCV(cv=5, estimator=LogisticRegression(random_state=42),
             param_grid=[{'C': [0.001, 0.01, 0.1, 1, 10, 100],
                          'penalty': ['l2'], 'solver': ['lbfgs', 'sag']},
                         {'C': [0.001, 0.01, 0.1, 1, 10, 100],
                          'penalty': ['l1', 'l2'],
                          'solver': ['liblinear', 'saga']}],
             scoring='roc_auc')

##Fitting Model

In [6]:


# Fit the model on the training data with cross-validation
log_reg_cv.fit(X_train_scaled, y_train)

# Get the best model with optimized hyperparameters
best_model = log_reg_cv.best_estimator_


##Predicting Using Model

In [8]:
# Predict on the test set using the best model
model = LogisticRegression()
model2 = RandomForestClassifier()
model2.fit(X_train_scaled,y_train)
model.fit(X_train_scaled,y_train)
y_pred = model.predict(X_test_scaled)
y_pred2 = model.predict(X_test_scaled)

##Evaluting Model

In [9]:
# Evaluate model performance with multiple metrics
accuracy = accuracy_score(y_test, y_pred)
accuracy2 = accuracy_score(y_test, y_pred2)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred)

print(f"Accuracy: {accuracy:.4f}")
print(f"Accuracy2: {accuracy2:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-score: {f1:.4f}")
print(f"ROC AUC: {roc_auc:.4f}")

Accuracy: 0.8000
Accuracy2: 0.8000
Precision: 0.9333
Recall: 0.5600
F1-score: 0.7000
ROC AUC: 0.7657


##Saving The Model

In [10]:
import joblib

# Save the trained model
joblib.dump(model2, 'heart_failure_model.pkl')

['heart_failure_model.pkl']

##Loading The model

In [ ]:
# Load the saved model
loaded_model = joblib.load('heart_failure_model.pkl')

# Prepare new data for prediction (ensure it has the same features as the training data)
new_data = ...  # Your new data

# Make predictions
predictions = loaded_model.predict(new_data)